In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
# Must set SNORKELDB before importing SnorkelSession
from snorkel import SnorkelSession
from snorkel.parser import TextDocPreprocessor
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence
session = SnorkelSession()

In [ ]:
doc_preprocessor = TextDocPreprocessor('small_data_pp/small_match_test.txt')

corpus_parser = CorpusParser()
corpus_parser.apply(doc_preprocessor)

Clearing existing...
Running UDF...


In [4]:
print "Documents:", session.query(Document).count()
print "Sentences:", session.query(Sentence).count()

Documents: 404
Sentences: 191058


In [2]:
from snorkel.models import candidate_subclass


GenePhenoPair = candidate_subclass('GenePhenoPair',['gene', 'pheno'])
#GenePhenoPair = candidate_subclass('GenePhenoPair',['gene'])

In [3]:
from snorkel.models import Document

all_sents = set()
docs = session.query(Document).order_by(Document.name).all()
for i, doc in enumerate(docs):
    if i > 0:break
    for s in doc.sentences:
        all_sents.add(s)

In [4]:
print len(all_sents)

376


In [5]:
from revised_extraction import GM, PM
from snorkel.candidates import Ngrams, CandidateExtractor

gene_ngrams = Ngrams(n_max=5)
pheno_ngrams = Ngrams(n_max=10)
cand_extractor = CandidateExtractor(GenePhenoPair, 
                                    [gene_ngrams, pheno_ngrams], [GM, PM],
                                    symmetric_relations=False)
#cand_extractor = CandidateExtractor(GenePhenoPair, 
                                    #[gene_ngrams], [GM],
                                    #symmetric_relations=False)

In [6]:
print "Extracting Candidates..."

cand_extractor.apply(all_sents, split=4)
all_cands = session.query(GenePhenoPair).filter(GenePhenoPair.split==4).all()
print "Number of dev candidates:", len(all_cands)

Extracting Candidates...
Clearing existing...
Running UDF...
[========================================] 100%

IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "gene_pheno_pair_gene_id_pheno_id_key"
DETAIL:  Key (gene_id, pheno_id)=(782942, 782287) already exists.
 [SQL: 'INSERT INTO gene_pheno_pair (id, gene_id, gene_cid, pheno_id, pheno_cid) VALUES (%(id)s, %(gene_id)s, %(gene_cid)s, %(pheno_id)s, %(pheno_cid)s)'] [parameters: ({'pheno_cid': None, 'gene_cid': None, 'pheno_id': 782287, 'id': 171158, 'gene_id': 782942}, {'pheno_cid': None, 'gene_cid': None, 'pheno_id': 782288, 'id': 171159, 'gene_id': 782942}, {'pheno_cid': None, 'gene_cid': None, 'pheno_id': 782288, 'id': 171160, 'gene_id': 782287}, {'pheno_cid': None, 'gene_cid': None, 'pheno_id': 783127, 'id': 171161, 'gene_id': 782313}, {'pheno_cid': None, 'gene_cid': None, 'pheno_id': 783155, 'id': 171162, 'gene_id': 782846}, {'pheno_cid': None, 'gene_cid': None, 'pheno_id': 782235, 'id': 171163, 'gene_id': 782930}, {'pheno_cid': None, 'gene_cid': None, 'pheno_id': 782206, 'id': 171164, 'gene_id': 78

In [3]:
from snorkel.viewer import SentenceNgramViewer
sv = SentenceNgramViewer(all_cands, session, annotator_name = 'gold')

<IPython.core.display.Javascript object>

In [4]:
sv